# Macroeconomics Landing → Bronze Pipeline

**Propósito**: Convierte datos CSV de indicadores macroeconómicos desde landing a formato Parquet en bronze layer

**Datasets**:
- Desempleo (IMF data)
- Inflación 12M 
- Tipo de cambio

**Funcionalidades**:
- Procesamiento automático de múltiples datasets
- Esquemas específicos por tipo de dato
- Particionado por fecha de proceso
- Manejo de encoding UTF-8

**Input**: `gs://bucket/lakehouse/landing/macroeconomics/*/dt=*/`
**Output**: `gs://bucket/lakehouse/bronze/macroeconomics/bronze_*_data/dt=*/`

In [ ]:
# ==============================================
# Macroeconomics landing -> bronze
# ==============================================
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StringType, DoubleType, DateType
from datetime import datetime

BUCKET = "dae-integrador-2025"
LANDING_BASE = f"gs://{BUCKET}/lakehouse/landing/macroeconomics"
BRONZE_BASE = f"gs://{BUCKET}/lakehouse/bronze/macroeconomics"

print(f"🔄 Iniciando Macroeconomics Landing → Bronze")
print(f"📥 Source: {LANDING_BASE}")
print(f"📤 Target: {BRONZE_BASE}")

In [ ]:
# Utilidades
def _list_subdirs(gcs_dir: str):
    """Lista subdirectorios usando Hadoop FileSystem"""
    jsc = sc._jsc
    hconf = jsc.hadoopConfiguration()
    Path = sc._gateway.jvm.org.apache.hadoop.fs.Path
    FileSystem = sc._gateway.jvm.org.apache.hadoop.fs.FileSystem
    fs = FileSystem.get(Path(gcs_dir).toUri(), hconf)
    return [st.getPath().getName() for st in fs.listStatus(Path(gcs_dir)) if st.isDirectory()]

def _pick_latest_dt_dir(base_dir: str):
    """Encuentra el directorio dt= más reciente"""
    subdirs = _list_subdirs(base_dir)
    dt_dirs = [d for d in subdirs if d.startswith("dt=")]
    if not dt_dirs:
        raise RuntimeError(f"No se encontraron directorios dt= en {base_dir}")
    latest = max(dt_dirs, key=lambda d: datetime.strptime(d.split("=")[1], "%Y-%m-%d"))
    return latest, latest.split("=")[1]

def process_dataset(dataset_name: str, schema: StructType):
    """Procesa un dataset específico"""
    print(f"\n📊 Procesando dataset: {dataset_name}")
    
    # Buscar último dt
    landing_path = f"{LANDING_BASE}/{dataset_name}"
    try:
        dt_dir, dt_str = _pick_latest_dt_dir(landing_path)
        csv_glob = f"{landing_path}/{dt_dir}/*.csv"
        print(f"📁 Último dt: {dt_dir}")
        print(f"📄 Leyendo: {csv_glob}")
    except Exception as e:
        print(f"⚠️ No se encontraron datos para {dataset_name}: {str(e)}")
        return
    
    # Leer CSV
    try:
        df_raw = (spark.read
                 .option("header", "true")
                 .option("delimiter", ",")
                 .option("encoding", "UTF-8")
                 .schema(schema)
                 .csv(csv_glob))
        
        if df_raw.rdd.isEmpty():
            print(f"⚠️ Dataset {dataset_name} está vacío")
            return
            
        row_count = df_raw.count()
        print(f"📊 Filas leídas: {row_count:,}")
        
    except Exception as e:
        print(f"❌ Error leyendo {dataset_name}: {str(e)}")
        return
    
    # Añadir metadatos
    df_bronze = (df_raw
                .withColumn("fecha_proceso", F.current_date())
                .withColumn("dt_captura", F.lit(dt_str))
                .withColumn("archivo_origen", F.input_file_name()))
    
    # Escribir a Bronze
    bronze_path = f"{BRONZE_BASE}/bronze_{dataset_name}_data"
    
    try:
        (df_bronze.write
         .mode("overwrite")
         .format("parquet")
         .partitionBy("dt_captura")
         .save(bronze_path))
        
        print(f"✅ {dataset_name} escrito en: {bronze_path}")
        
    except Exception as e:
        print(f"❌ Error escribiendo {dataset_name}: {str(e)}")
        return

In [ ]:
# Esquemas por dataset
schemas = {
    "desempleo_imf": StructType([
        StructType().add("fecha", StringType(), True)
                   .add("pais", StringType(), True)
                   .add("tasa_desempleo", StringType(), True)
                   .add("fuente", StringType(), True)
    ]),
    
    "inflacion_12m": StructType([
        StructType().add("fecha", StringType(), True)
                   .add("pais", StringType(), True)
                   .add("inflacion_anual", StringType(), True)
                   .add("indice_base", StringType(), True)
    ]),
    
    "tipo_cambio": StructType([
        StructType().add("fecha", StringType(), True)
                   .add("moneda_origen", StringType(), True)
                   .add("moneda_destino", StringType(), True)
                   .add("tasa_cambio", StringType(), True)
                   .add("tipo_cambio", StringType(), True)
    ])
}

print("📋 Esquemas definidos para datasets:")
for dataset in schemas.keys():
    print(f"  - {dataset}")

In [ ]:
# Procesar todos los datasets
print("🚀 Iniciando procesamiento de datasets...")

for dataset_name, schema in schemas.items():
    process_dataset(dataset_name, schema)

print("\n🎉 ¡Macroeconomics Bronze ingestion completada!")

In [ ]:
# Verificación final
print("\n📋 Verificación de datos en Bronze:")

for dataset_name in schemas.keys():
    bronze_path = f"{BRONZE_BASE}/bronze_{dataset_name}_data"
    try:
        df_check = spark.read.parquet(bronze_path)
        count = df_check.count()
        print(f"  ✅ {dataset_name}: {count:,} filas")
    except Exception as e:
        print(f"  ❌ {dataset_name}: Error - {str(e)}")